# CURRENCY DIMENSION

In [1]:
import pandas as pd
from connection_script import connect_databases

In [2]:
db_op, db_etl = connect_databases()

## EXTRACT

In [3]:
dim_currency = pd.read_sql_query('SELECT * FROM [Sales].[Currency]', db_op)

## TRANSFORM

In [4]:
# Drop columns
dim_currency = dim_currency.drop(columns='ModifiedDate')

In [5]:
# Rename Columns
dim_currency = dim_currency.rename(columns={"CurrencyCode": "CurrencyAlternateKey", "Name": "CurrencyName"})

In [6]:
# Sort registries
dim_currency = dim_currency.assign(LowerCurrencyName=list(dim_currency["CurrencyName"]))
dim_currency["LowerCurrencyName"] = dim_currency["LowerCurrencyName"].apply(lambda x: x.lower())
dim_currency = dim_currency.sort_values(by=["LowerCurrencyName"])
dim_currency = dim_currency.reset_index(drop=True)
dim_currency.index += 1

In [7]:
dim_currency = dim_currency.drop(columns=["LowerCurrencyName"])
dim_currency

,CurrencyAlternateKey,CurrencyName
1,AFA,Afghani
2,DZD,Algerian Dinar
3,ARS,Argentine Peso
4,AMD,Armenian Dram
5,AWG,Aruban Guilder
...,...,...
101,KRW,Won
102,JPY,Yen
103,CNY,Yuan Renminbi
104,ZWD,Zimbabwe Dollar


## Load

In [8]:
dim_currency.to_sql('DimCurrency', db_etl, if_exists='replace', index_label='CurrencyKey')

105